In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
from tflearn.data_utils import shuffle, to_categorical
import numpy as np

In [2]:
# Load kaggle train data
train = (pd.read_csv('./input/train.csv')).astype('float32')
train.shape

(42000, 785)

In [3]:
# extract X, y kaggle train data
X, Y = train.drop('label', axis=1).values, train['label'].values

print X.shape
print Y.shape

(42000, 784)
(42000,)


In [4]:
X, X_test, Y, Y_test = train_test_split(
    X,
    Y,
    test_size=0.15,
    random_state=42,
    stratify=train[train.columns[0]].values, # to preserve initial class balance
)

print "X shape:", X.shape
print "X_test shape:", X_test.shape
print "Y shape:", Y.shape
print "Y_test shape:", Y_test.shape

X shape: (35700, 784)
X_test shape: (6300, 784)
Y shape: (35700,)
Y_test shape: (6300,)


In [5]:
# Normalization
X = X / 255
X_test = X_test / 255

In [6]:
X = X.reshape([-1, 28, 28, 1])
X_test = X_test.reshape([-1, 28, 28, 1])
Y = to_categorical(Y, 10)
Y_test = to_categorical(Y_test, 10)

print "X shape:", X.shape
print "X_test shape:", X_test.shape
print "Y shape:", Y.shape
print "Y_test shape:", Y_test.shape

X shape: (35700, 28, 28, 1)
X_test shape: (6300, 28, 28, 1)
Y shape: (35700, 10)
Y_test shape: (6300, 10)


## Train

In [8]:
OPTIMIZER = 'Adam'
LEARNING_RATE = 0.0001
N_EPOCH = 10

RUN_ID = 'Train Adam, 0.0001, n10'

network = input_data(shape=[None, 28, 28, 1], name='input')

network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)

network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)

network = fully_connected(network, 128, activation='tanh')
network = dropout(network, 0.8)

network = fully_connected(network, 256, activation='tanh')
network = dropout(network, 0.8)

network = fully_connected(network, 10, activation='softmax')

network = regression(
    network, 
    optimizer=OPTIMIZER, 
    learning_rate=LEARNING_RATE, 
    loss='categorical_crossentropy', 
    name='target'
)

model = tflearn.DNN(network, tensorboard_verbose=3)
model.fit(
    {'input': X}, {'target': Y},
    n_epoch=N_EPOCH,
    validation_set=({'input': X_test}, {'target': Y_test}),
    snapshot_step=100, 
    show_metric=True,
    run_id=RUN_ID
)

Training Step: 5579  | total loss: 0.14418 | time: 26.690s
| Adam | epoch: 010 | loss: 0.14418 - acc: 0.9817 -- iter: 35648/35700
Training Step: 5580  | total loss: 0.13183 | time: 27.776s
| Adam | epoch: 010 | loss: 0.13183 - acc: 0.9821 | val_loss: 0.04915 - val_acc: 0.9852 -- iter: 35700/35700
--


## Predict

In [9]:
X_submission = (pd.read_csv('./input/test.csv').values).astype('float32')
X_submission = X_submission / 255 # Normalization

print X_submission.shape

(28000, 784)


In [10]:
X_submission = X_submission.reshape([-1, 28, 28, 1])

print X_submission.shape

(28000, 28, 28, 1)


In [11]:
Y_predict = model.predict(X_submission)
Y_predict = np.argmax(Y_predict,axis=1)
print Y_predict[0]
print Y_predict[1]
print Y_predict[2]
print Y_predict[3]
print Y_predict[4]

2
0
9
9
3


In [12]:
submission=pd.DataFrame({"ImageId": list(range(1,len(Y_predict)+1)), "Label": Y_predict})
submission.to_csv("./submissions/submission_ccn_tflearn_lr_0_001_nepoch_10_normalization.csv", index=False, header=True)